In [24]:
from bs4 import BeautifulSoup
import requests
import string
import pandas as pd

In [25]:
candidates_2017 = "https://info.kingcounty.gov/kcelections/Vote/contests/candidates.aspx?eid=9"
listOfRaces = [candidates_2017, ]

In [32]:
def stripPunc(s):
    if s is not None:
        table = str.maketrans({key: None for key in string.punctuation})
        return s.translate(table)
    else:
        return s

In [27]:
def extractCandidateInfo(c):
    info = c.find('span', {'class': 'ballotname'})
    name = info.text
    cid = info["id"]
    
    link = c.find('a', {'class': 'candidate-anchor'})["href"]
    html = getattr(requests.get(link), 'content', None)
    if html is not None:
        soup = BeautifulSoup(html, 'html.parser')
        content = ' '.join([getattr(x, 'text', None) for x in soup.findAll('p')]).strip()
    
    party = getattr(c.find('span', {'class': 'candidateparty'}), 'text', None)
    
    return dict(candidate_name = name, 
                candidate_id = cid, 
                candidate_party = party,
                candidate_url = link,
                candidate_info = content)

In [33]:
def extractAllCandidates(soup):
    
    data = []
    
    district = stripPunc(getattr(soup.find('span', {'class': 'sp-bt-title-1'}), 'text', None))
    race = stripPunc(getattr(soup.find('span', {'class': 'sp-bt-title-2'}), 'text', None))
    
    candidates = soup.findAll('li', {'class': 'li-candidate'})
    
    for c in candidates:
        candidate = extractCandidateInfo(c)
        candidate['district'] = district
        candidate['race'] = race
        data.append(candidate)
    
    return data

In [35]:
candidateDF = []
for race in listOfRaces:
    page = requests.get(race)
    soup = BeautifulSoup(page.content, 'html.parser')

    candidateList = soup.find('div', {'class': 'candidate-list-group', 'id': 'State'}).findAll('div', {'class': 'candidatelist-div'})
    candidates = [candidate for sublist in [extractAllCandidates(html) for html in candidateList] for candidate in sublist]

    candidateDF.append(pd.concat([pd.DataFrame(x, index=[x['candidate_id'], ]) for x in candidates]))

In [36]:
allCandidates = pd.concat(candidateDF)
allCandidates.to_pickle('../data/allCandidates.pkl')